<a href="https://colab.research.google.com/github/EfthymiosCosta/Neural-Networks-Weight-Uncertainty/blob/main/Bayes_by_Backprop_NN_400_layers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/kumar-shridhar/PyTorch-BayesianCNN.git

remote: Total 1687 (delta 71), reused 88 (delta 46), pack-reused 1561
Receiving objects: 100% (1687/1687), 67.83 MiB | 3.16 MiB/s, done.
Resolving deltas: 100% (1007/1007), done.


In [2]:
%cd "PyTorch-BayesianCNN"

/content/PyTorch-BayesianCNN


In [3]:
from __future__ import print_function

import os
import argparse

import torch
import numpy as np
from torch.optim import Adam, lr_scheduler
from torch.nn import functional as F

import data
import utils
import metrics
import config_bayesian as cfg
from models.BayesianModels.Bayesian3Conv3FC import BBB3Conv3FC
from models.BayesianModels.BayesianAlexNet import BBBAlexNet
from models.BayesianModels.BayesianLeNet import BBBLeNet

In [12]:
import math
import torch.nn as nn
from layers import BBB_Linear, BBB_Conv2d
from layers import BBB_LRT_Linear, BBB_LRT_Conv2d
from layers import FlattenLayer, ModuleWrapper


class Net(ModuleWrapper):
    '''The architecture of LeNet with Bayesian Layers'''

    def __init__(self, outputs, inputs, priors, layer_type='lrt', activation_type='softplus'):
        super(Net, self).__init__()

        self.num_classes = outputs
        self.layer_type = layer_type
        self.priors = priors

        BBBLinear = BBB_Linear
        self.act = nn.ReLU

        self.flatten = FlattenLayer(1024)
        self.fc1 = BBBLinear(1024, 400, bias=True, priors=self.priors)
        self.act1 = self.act()

        self.fc2 = BBBLinear(400, 400, bias=True, priors=self.priors)
        self.act2 = self.act()

        self.fc3 = BBBLinear(400, outputs, bias=True, priors=self.priors)

In [5]:
# CUDA settings
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def getModel(net_type, inputs, outputs, priors, layer_type, activation_type):
    return Net(outputs, inputs, priors, layer_type, activation_type)

In [6]:
def train_model(net, optimizer, criterion, trainloader, num_ens=1, beta_type=0.1, epoch=None, num_epochs=None):
    net.train()
    training_loss = 0.0
    accs = []
    kl_list = []
    for i, (inputs, labels) in enumerate(trainloader, 1):

        optimizer.zero_grad()

        inputs, labels = inputs.to(device), labels.to(device)
        outputs = torch.zeros(inputs.shape[0], net.num_classes, num_ens).to(device)

        kl = 0.0
        for j in range(num_ens):
            net_out, _kl = net(inputs)
            kl += _kl
            outputs[:, :, j] = F.log_softmax(net_out, dim=1)
        
        kl = kl / num_ens
        kl_list.append(kl.item())
        log_outputs = utils.logmeanexp(outputs, dim=2)

        beta = metrics.get_beta(i-1, len(trainloader), beta_type, epoch, num_epochs)
        loss = criterion(log_outputs, labels, kl, beta)
        loss.backward()
        optimizer.step()

        accs.append(metrics.acc(log_outputs.data, labels))
        training_loss += loss.cpu().data.numpy()
    return training_loss/len(trainloader), np.mean(accs), np.mean(kl_list)


In [7]:
def validate_model(net, criterion, validloader, num_ens=1, beta_type=0.1, epoch=None, num_epochs=None):
    """Calculate ensemble accuracy and NLL Loss"""
    net.train()
    valid_loss = 0.0
    accs = []

    for i, (inputs, labels) in enumerate(validloader):
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = torch.zeros(inputs.shape[0], net.num_classes, num_ens).to(device)
        kl = 0.0
        for j in range(num_ens):
            net_out, _kl = net(inputs)
            kl += _kl
            outputs[:, :, j] = F.log_softmax(net_out, dim=1).data

        log_outputs = utils.logmeanexp(outputs, dim=2)

        beta = metrics.get_beta(i-1, len(validloader), beta_type, epoch, num_epochs)
        valid_loss += criterion(log_outputs, labels, kl, beta).item()
        accs.append(metrics.acc(log_outputs, labels))

    return valid_loss/len(validloader), np.mean(accs)


In [8]:
def getDataloader(trainset, testset, batch_size, num_workers):
    num_train = len(trainset)

    train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle = True, num_workers=num_workers)
    test_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size, 
        num_workers=num_workers)

    return train_loader, test_loader

In [14]:
def run(dataset, net_type):

    ELBO = []
    train_acc_tracker = []
    val_acc_tracker = []
    # Hyper Parameter settings
    layer_type = cfg.layer_type
    activation_type = cfg.activation_type
    priors = cfg.priors

    train_ens = cfg.train_ens
    valid_ens = cfg.valid_ens
    n_epochs = 200
    lr_start = cfg.lr_start
    # set to 0 cos otherwise sometimes get assertion error: can only test a child process
    num_workers = 2
    valid_size = 0.001
    batch_size = cfg.batch_size
    beta_type = cfg.beta_type

    trainset, testset, inputs, outputs = data.getDataset(dataset)
    train_loader, test_loader = getDataloader(
        trainset, testset, batch_size, num_workers)
    net = getModel(net_type, inputs, outputs, priors, layer_type, activation_type).to(device)

    ckpt_dir = f'checkpoints/{dataset}/bayesian'
    ckpt_name = f'checkpoints/{dataset}/bayesian/model_{net_type}_{layer_type}_{activation_type}.pt'

    if not os.path.exists(ckpt_dir):
        os.makedirs(ckpt_dir, exist_ok=True)

    criterion = metrics.ELBO(len(trainset)).to(device)
    optimizer = Adam(net.parameters(), lr=lr_start)
    lr_sched = lr_scheduler.ReduceLROnPlateau(optimizer, patience=6, verbose=True)
    valid_loss_max = np.Inf
    for epoch in range(n_epochs):  # loop over the dataset multiple times

        train_loss, train_acc, train_kl = train_model(net, optimizer, criterion, train_loader, num_ens=train_ens, beta_type=beta_type, epoch=epoch, num_epochs=n_epochs)
        # changed valid LOSS TO TEST LOSS!!!
        valid_loss, valid_acc = validate_model(net, criterion, test_loader, num_ens=valid_ens, beta_type=beta_type, epoch=epoch, num_epochs=n_epochs)
        lr_sched.step(valid_loss)

        ELBO += [train_loss]
        train_acc_tracker += [train_acc]
        val_acc_tracker += [valid_acc]

        print('Epoch: {} \tTraining Loss: {:.4f} \tTraining Accuracy: {:.4f} \tValidation Loss: {:.4f} \tValidation Accuracy: {:.4f} \ttrain_kl_div: {:.4f}'.format(
            epoch, train_loss, train_acc, valid_loss, valid_acc, train_kl))

        # save model if validation accuracy has increased
        if valid_loss <= valid_loss_max:
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
                valid_loss_max, valid_loss))
            torch.save(net.state_dict(), ckpt_name)
            valid_loss_max = valid_loss
    return net, ELBO, train_acc_tracker, val_acc_tracker

In [10]:
dataset = 'MNIST'
net_type = 'Net'

In [15]:
net, ELBO, train_acc, test_acc = run(dataset, net_type)

Epoch: 0 	Training Loss: 7001218.3787 	Training Accuracy: 0.8759 	Validation Loss: 4548794.0250 	Validation Accuracy: 0.9113 	train_kl_div: 69730427.8468
Validation loss decreased (inf --> 4548794.025000).  Saving model ...
Epoch: 1 	Training Loss: 3779287.3064 	Training Accuracy: 0.9097 	Validation Loss: 3176680.0312 	Validation Accuracy: 0.9147 	train_kl_div: 37576121.0723
Validation loss decreased (4548794.025000 --> 3176680.031250).  Saving model ...
Epoch: 2 	Training Loss: 2758580.1436 	Training Accuracy: 0.9178 	Validation Loss: 2389482.8125 	Validation Accuracy: 0.9280 	train_kl_div: 27394662.3149
Validation loss decreased (3176680.031250 --> 2389482.812500).  Saving model ...
Epoch: 3 	Training Loss: 2108256.2617 	Training Accuracy: 0.9272 	Validation Loss: 1852714.9406 	Validation Accuracy: 0.9377 	train_kl_div: 20914604.0681
Validation loss decreased (2389482.812500 --> 1852714.940625).  Saving model ...
Epoch: 4 	Training Loss: 1652309.9021 	Training Accuracy: 0.9317 	Valid

In [16]:
import matplotlib.pyplot as plt

In [ ]:
%cd /content

In [ ]:
from google.colab import drive
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/Colab Notebooks/'
PATH = 'gdrive/MyDrive/Colab Notebooks/Project_StatML_Module2/'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import pickle

In [ ]:
%cd gdrive

[Errno 2] No such file or directory: 'gdrive'
/content/BayesianCNN


In [ ]:
with open(PATH + "LeNET_ELBO_evolution.pickle", 'wb') as f:
    pickle.dump(ELBO, f)

In [ ]:
with open(PATH + "LeNET_train_acc_evolution.pickle", 'wb') as f:
    pickle.dump(train_acc, f)

In [ ]:
with open(PATH + "LeNET_test_acc_evolution.pickle", 'wb') as f:
    pickle.dump(test_acc, f)